# **Connect With Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# **Download Yolov5 From Github**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

# **Prepare Environment**

In [ ]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# **Testing Time**

### **Access Weights and Input Images**

In [ ]:
# weightPath = "../gdrive/MyDrive/finalModel/best.pt"
# imagePath = "../gdrive/MyDrive/testingImage/"

In [ ]:
%cd /content/yolov5/
!python detect.py --weights ../gdrive/MyDrive/finalModel/best.pt --img 416 --conf 0.5 --save-txt --save-conf --source ../gdrive/MyDrive/testingImage/

### **Calculate Final Outputs**

In [ ]:
import glob
import cv2 as cv
from IPython.display import Image, display

def indexToClass(classIndex):
  if(classIndex == str(0)): return int(1)
  elif(classIndex == str(1)): return int(10)
  elif(classIndex == str(2)): return int(5)
  elif(classIndex == str(3)): return int(50)
  elif(classIndex == str(4)): return int(0)

def calculation(imagePath):
  for num in range(len(imagePath)):
    resultPath = imagePath[num].replace("exp/", "exp/labels/")
    currentResultPath = resultPath[0:-3]+"txt"
    f = open(currentResultPath, 'r') # open single file
    print("Image %s" %(imagePath[num]))

    classIndex, boundingBox = [], []
    for line in f.readlines():
      index = line.find(" ")
      classIndex.append(line[0:index])
      boundingBox.append(line[index+1:-1].split()) # split class and bounding box location

    difference = []
    for i in range(len(boundingBox)):     # detecting overlapping bounding boxes
      difference.append([1] * (len(boundingBox)))
      for j in range(len(boundingBox)):
        if(abs(float(boundingBox[i][0])-float(boundingBox[j][0]))>0.05 or abs(float(boundingBox[i][1])-float(boundingBox[j][1]))>0.05): difference[i][j] = 1
        else: difference[i][j] = 0

    for i in range(len(boundingBox)):
      for j in range(len(boundingBox)): 
        if(i!=j and not difference[i][j]): # compare with the confidence
          if(boundingBox[i][4]>boundingBox[j][4] and (classIndex[i]!="4" and classIndex[j]!="4")):
            print("\tOverlap Detected: Select Class %d with Confidence %s Discard Class %d with Confidence %s" %(indexToClass(classIndex[i]), boundingBox[i][4], indexToClass(classIndex[j]), boundingBox[j][4]))
            classIndex[j] = str(4)
          elif(boundingBox[i][4]<=boundingBox[j][4] and (classIndex[i]!="4" and classIndex[j]!="4")): 
            print("\tOverlap Detected: Select Class %d with Confidence %s Discard Class %d with Confidence %s" %(indexToClass(classIndex[j]), boundingBox[j][4], indexToClass(classIndex[i]), boundingBox[i][4]))
            classIndex[i] = str(4) 

    totalDollar = 0 # counting total dollars
    for coin in classIndex:
      totalDollar += indexToClass(coin)
    
    img = cv.imread(imagePath[num])
    height, width = img.shape[:2]
    ratio = width/height
    resizeImg = cv.resize(img, (int(728*ratio), 728), interpolation=cv.INTER_AREA)
    newHeight, newWidth = resizeImg.shape[:2]
    font, bottomLeftCornerOfText, fontScale, fontColor, lineScale= cv.FONT_HERSHEY_DUPLEX, (10,int(newHeight*0.98)), 1, (0,255,255), 2
    cv.putText(resizeImg, "Total Dollar: "+str(totalDollar), bottomLeftCornerOfText, font, fontScale, fontColor, lineScale)
    cv.imwrite('output.jpg', resizeImg) 
    
    display(Image(filename='output.jpg'))
    print("Total Dollar: %d\n" %(totalDollar))

In [ ]:
resultPath = glob.glob('/content/yolov5/runs/detect/exp/labels/*.txt') # may have to change the path
imagePath = glob.glob('/content/yolov5/runs/detect/exp/*.jpg') # may have to change the path
calculation(imagePath)